In [1]:
import requests
from bs4 import BeautifulSoup
URL='https://ca.trustpilot.com/review/approvalgenie.ca'
resp = requests.get(URL)
soup = BeautifulSoup(resp.text, 'html.parser')
print(soup.title)

<title>Approval Genie Reviews | Read Customer Service Reviews of approvalgenie.ca</title>


## 1. Use the requests module to download the html for URL.

In [9]:
# check what the response contains
type(resp)
print(dir(resp))

print(resp.status_code)
# HTTPS status codes:
# https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

print(resp.text)
# This is the html source
html_code = resp.text


['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']
200
<!DOCTYPE html>
<html lang="en-ca">
<head>
    

        <script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js"
            type="application/javascript"
            charset="UTF-8"
            da

## 2. Extract the total number of reviews.

In [3]:
import csv 
soup = BeautifulSoup(resp.text, 'html.parser')

# Find reviews count

#<h2 class="headline">Reviews&nbsp;&nbsp;<spanclass="headline__review-count">1,292</span></h2>
reviews_count_html = soup.findAll('h2', {'class': 'headline'})[0]
reviews_count_html

<h2 class="headline">Reviews  <span class="headline__review-count">796</span></h2>

In [4]:
total_review_count=int(reviews_count_html.span.text.replace(',', ''))
print("### Total review count ### " + str(total_review_count))

796


## 3. Iterate over the review pages.
## 4. From each page extract the reviews.


In [5]:
import math
reviews_perpage =20
num_pages= math.ceil(total_review_count/reviews_perpage)+1
domain_name='https://ca.trustpilot.com'
start_url='https://ca.trustpilot.com/review/approvalgenie.ca'
name = 'Approval Genie'
url = start_url
store_review = []
print("###########Loading Content from Websites############"")
for i in range(num_pages):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    reviews_html = soup.findAll('div', {'class': 'review-content'})
    
    
    for each_review in reviews_html:
        review_date = each_review.script.text.split('"')[3]
        review_rating = int(each_review.img['alt'].split(' ')[0])
        if each_review.p:
            review_text= each_review.p.text
        else:
            review_text=''
        review_title = each_review.h2.a.text
        review_body = review_title + review_text
        store_review.append([name,review_date,review_rating,review_body])
    if soup.findAll('a', {'class': 'next-page'}):
        next_page_link = soup.findAll('a', {'class': 'next-page'})[0]['href']
        url=domain_name+next_page_link
    else:
        print("###########Finished############"")
        break


## 5.  From each review, store the following to the CSV file:



In [6]:
import pandas as pd
df = pd.DataFrame(store_review)


,0,1,2,3
0,Approval Genie,2021-05-26T00:17:26+00:00,5,\n Wish Granted!\n ...
1,Approval Genie,2021-05-25T20:46:28+00:00,5,\n I thought getting an...
2,Approval Genie,2021-05-25T17:42:33+00:00,5,\n Best service out her...
3,Approval Genie,2021-05-25T03:40:22+00:00,5,\n We Got what we reall...
4,Approval Genie,2021-05-23T11:36:26+00:00,5,\n I needed a car and a...
...,...,...,...,...
712,Approval Genie,2019-12-13T16:58:13+00:00,4,\n I'm satisfied there...
713,Approval Genie,2019-12-13T16:56:15+00:00,5,\n Emil who served me\n...
714,Approval Genie,2019-12-13T16:56:12+00:00,5,\n Nothing short of ext...
715,Approval Genie,2019-12-13T16:55:52+00:00,5,\n They worked with me ...


In [7]:
df=df.rename(columns={0: "comapnyName", 1: "datePublished",2:"ratingValue",3:"reviewBody"})


,comapnyName,datePublished,ratingValue,reviewBody
0,Approval Genie,2021-05-26T00:17:26+00:00,5,\n Wish Granted!\n ...
1,Approval Genie,2021-05-25T20:46:28+00:00,5,\n I thought getting an...
2,Approval Genie,2021-05-25T17:42:33+00:00,5,\n Best service out her...
3,Approval Genie,2021-05-25T03:40:22+00:00,5,\n We Got what we reall...
4,Approval Genie,2021-05-23T11:36:26+00:00,5,\n I needed a car and a...
...,...,...,...,...
712,Approval Genie,2019-12-13T16:58:13+00:00,4,\n I'm satisfied there...
713,Approval Genie,2019-12-13T16:56:15+00:00,5,\n Emil who served me\n...
714,Approval Genie,2019-12-13T16:56:12+00:00,5,\n Nothing short of ext...
715,Approval Genie,2019-12-13T16:55:52+00:00,5,\n They worked with me ...


In [8]:
df.to_csv(name + '_result.csv')
print("###########Save CSV Success############"")
